In [16]:
using Base.Test
using BenchmarkTools

In [2]:
import TypedPolynomials

In [77]:
reload("TypedPolynomials")
t = TypedPolynomials

TypedPolynomials

In [95]:
module sp

using TypedPolynomials: mergesorted
import TypedPolynomials: AbstractVariable,
    AbstractMonomial,
    AbstractTerm,
    AbstractPolynomial,
    name,
    exponents,
    exponent,
    variables

import Base: literal_pow, +, -, *, /, ==, isless,
copy, promote_rule, convert

export @polyvar,
       Variable,
       Monomial,
       Term,
       Polynomial,
       name,
       exponents,
       variables,
       coefficient,
       monomial,
       terms,
       degree,
       subs

struct Variable{Name} <: AbstractVariable
end

copy(v::Variable) = v

name(v::Variable{Name}) where {Name} = Name

macro polyvar(names...)
    exprs = [
        quote
            $(esc(name)) = Variable{$(esc(Expr(:quote, name)))}()
        end
        for name in names
    ]
    Expr(:block, exprs...)
end

struct Power{Var, Exponent}
    function Power{V, E}() where {V, E}
        new{typeassert(V, Variable), typeassert(E, Int)}()
    end
end

variable(p::Power{V}) where {V} = V
exponent(p::Power{V, E}) where {V, E} = E

struct Monomial{Powers} <: AbstractMonomial
    function Monomial{P}() where {P}
        new{typeassert(P, Tuple{Vararg{Power}})}()
    end
end

variables(m::Monomial{Powers}) where {Powers} = variable.(Powers)
exponents(m::Monomial{Powers}) where {Powers} = exponent.(Powers)
powers(::Type{Monomial{Powers}}) where {Powers} = Powers
powers(m::Monomial) = powers(typeof(m))

struct Term{M <: Monomial, T} <: AbstractTerm
    coefficient::T
end

struct Polynomial{Terms <: Tuple{Vararg{Term}}}
    terms::Terms
end

function literal_pow(^, v::V, ::Type{Val{x}}) where {V <: Variable, x}
    Monomial{(Power{v, x}(),)}()
end

const MonomialLike = Union{<:Variable, <:Monomial}
const TermLike = Union{<:MonomialLike, <:Term}
const PolynomialLike = Union{<:TermLike, <:Polynomial}

promote_rule(::Type{<:Variable}, ::Type{<:Variable}) = Monomial
promote_rule(::Type{<:Variable}, ::Type{<:Monomial}) = Monomial

convert(::Type{Monomial}, v::Variable) = Monomial{(Power{v, 1}(),)}()


(*)(v1::V, v2::V) where {V <: Variable} = Monomial{(Power{v1, 2}(),)}()
@generated function (*)(::Power{V, E1}, ::Power{V, E2}) where {V, E1, E2}
    :(Power{V, $(E1 + E2)}())
end
@generated function (*)(m1::Monomial, m2::Monomial)
    newpowers = mergesorted(
        collect(powers(m1)), 
        collect(powers(m2)), 
        (p1, p2) -> name(variable(p1)) < name(variable(p2)), 
        *)
    :(Monomial{$(Tuple(newpowers))}())
end     

for op in [:+, :*, :-, :(==)]
    @eval $op(p1::PolynomialLike, p2::PolynomialLike) = $op(promote(p1, p2)...)
    @eval $op(p::PolynomialLike, x) = $op(promote(p, x)...)
    @eval $op(x, p::PolynomialLike) = $op(promote(x, p)...)
end

(==)(v1::V, v2::V) where {V <: Variable} = true
(==)(v1::Variable, v2::Variable) = false
function (==)(m1::Monomial, m2::Monomial)
    v1 = variables(m1)
    e1 = exponents(m1)
    v2 = variables(m2)
    e2 = exponents(m2)
    i1 = 1
    i2 = 1
    while true
        while i1 <= length(e1) && e1[i1] == 0
            i1 += 1
        end
        while i2 <= length(e2) && e2[i2] == 0
            i2 += 1
        end
        if i1 > length(e1) && i2 > length(e1)
            return true
        elseif i1 > length(e1) || i2 > length(e2)
            return false
        elseif v1[i1] != v2[i2]
            return false
        elseif e1[i1] != e2[i2]
            return false
        else
            i1 += 1
            i2 += 1
        end
    end
end



module tests

using Base.Test
using sp
using sp: Power

macro wrappedallocs(expr)
    @assert expr.head == :call
    f = expr.args[1]
    args = expr.args[2:end]
    argnames = [gensym() for a in args]
    quote
        function g($(argnames...))
            @allocated $(Expr(:call, esc(f), argnames...))
        end
        $(Expr(:call, :g, [esc(a) for a in args]...))
    end
end


@testset "variables" begin
    @polyvar x y
    
    @test x * x == x^2
    @test x^1 == x
#     @test x^0 == 1
    @test (@wrappedallocs x * x) == 0
    @test (@wrappedallocs x^2) == 0
    @test (@wrappedallocs x^1) == 0
#     @test (@wrappedallocs x^0) == 0 
end

@testset "orderings" begin
    @polyvar x y z
    
#     @test Monomial(y) < Monomial(x)
#     @test Monomial(x) > Monomial(y)
#     @test Monomial(y) < x
#     @test y < Monomial(x)
    @test y < x
    @test x > y
#     @test x > Monomial(y)
#     @test Monomial(x) > y
end

@testset "monomials" begin
    @polyvar x y z
    
    @test typeof(@inferred x * x) == Monomial{(Power{x, 2}(),)}
    @test typeof(@inferred x * y) == Monomial{(Power{x, 1}(), Power{y, 1}())}
    @test typeof(@inferred y * x) == Monomial{(Power{x, 1}(), Power{y, 1}())}
    @test typeof(@inferred x * z * y) == Monomial{(Power{x, 1}(), Power{y, 1}(), Power{z, 1}())}
end

end

end

sp.@polyvar x y z

Test Summary: | Pass  Total
variables     |    5      5
Test Summary: | 

Pass  Total
orderings     |    2      2
Test Summary: | Pass  Total
monomials     |    4      4


z

In [92]:
sp.Monomial(x)

x

In [94]:
[x, y]

2-element Array{sp.Monomial,1}:
 x
 y

In [90]:
x * y * z

xyz

In [87]:
typeof(x*y)

sp.Monomial{(sp.Power{x,1}(), sp.Power{y,1}())}

In [73]:
x * x

x^2

In [143]:
x^2

x^2

In [144]:
@benchmark $x * $x

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     0.023 ns (0.00% GC)
  median time:      0.037 ns (0.00% GC)
  mean time:        0.043 ns (0.00% GC)
  maximum time:     0.543 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [137]:
typeof(x^2)

sp.Monomial{(sp.Power{x,2}(),)}

In [138]:
@eval @code_warntype $(expand(:(x^2)))

Variables:
  #self#::Base.#literal_pow
  ^::Base.#^
  v::sp.Variable{:x}
  #unused#::Any

Body:
  begin 
      return $(QuoteNode(x^2))
  end::sp.Monomial{(sp.Power{x,2}(),)}


In [139]:
@benchmark $x^2

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     0.024 ns (0.00% GC)
  median time:      0.036 ns (0.00% GC)
  mean time:        0.042 ns (0.00% GC)
  maximum time:     0.185 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [71]:
x^2

x^2

In [43]:
typeof(m)

sp.Monomial{(sp.Power{sp.Variable{:x}}(2),)}

In [44]:
@code_warntype sp.exponents(m)

Variables:
  #self#::TypedPolynomials.#exponents
  m::sp.Monomial{(sp.Power{sp.Variable{:x}}(2),)}

Body:
  begin 
      return (Core.tuple)((Core.getfield)((Base.getfield)($(Expr(:static_parameter, 1)), 1)::sp.Power{sp.Variable{:x}}, :exponent)::Int64)::Tuple{Int64}
  end::Tuple{Int64}


In [45]:
@benchmark sp.variables($m)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     0.037 ns (0.00% GC)
  median time:      0.040 ns (0.00% GC)
  mean time:        0.049 ns (0.00% GC)
  maximum time:     8.393 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [46]:
sp.variables(m)

(x,)

In [16]:
@which t.name(x)

name(::TypedPolynomials.AbstractVariable) in TypedPolynomials at /Users/rdeits/.julia/v0.6/TypedPolynomials/src/abstracttypes.jl:6

In [9]:
sp.degree(x)

LoadError: [91mUndefVarError: degree not defined[39m

In [3]:
t.shortest_common_supersequence((:x,), (:y,))

2-element Array{Any,1}:
 :y
 :x